In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

# Pandas Row-Column Transformations

There comes a time in the life of any data scientist when he or she needs to transform the set of columns in a dataset into rows and vice versa.

This is not a common operation, but it does happen every now and then. Pandas has two set of methods to do this:

* stack and unstack
* pivot and melt

Again these sets of methods basically do the same thing.


I have found that stack and unstack are much more stable but a bit less powerful. So those are the ones I use. 

Right at the end we will go over pandas dummy variables being the last way to make the transformation. 

Check out the full documentation for both [stack and unstack](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html) and [dummy variables](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#computing-indicator-dummy-variables), but be warned it is a bit long :)

Okay Let's get started

In [2]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


A question we might want to ask is: what is the male to female ratio on different days of the week?

To do this we might start with a groupby:

In [4]:
tips_gb = tips.groupby(['day', 'sex']).agg({'size': 'sum'})
tips_gb

size
day  sex         
Thur Male      73
     Female    79
Fri  Male      21
     Female    19
Sat  Male     156
     Female    63
Sun  Male     163
     Female    53

So we are getting somewhere, but it is a bit hard to tell the number of male and female visitors by looking at it, and you might want to do more columnwise operations comparing the male to the female visitors.

So what you might want to do is take the values in the column sex and make them into column. This is where unstacking comes in!

## Unstack

In [14]:
tips_us = tips_gb.unstack()
tips_us

size       
sex  Male Female
day             
Thur   73     79
Fri    21     19
Sat   156     63
Sun   163     53

Notice we basically moved an index to the columns!

In [15]:
# you could do the same with the days of the week
tips_gb.unstack(0)

size              
day    Thur Fri  Sat  Sun
sex                      
Male     73  21  156  163
Female   79  19   63   53

The problem is that now we have this odd new object as the columns:

In [16]:
tips_us.columns

MultiIndex(levels=[['size'], ['Male', 'Female']],
           codes=[[0, 0], [0, 1]],
           names=[None, 'sex'])

And while you can do things with it:

In [17]:
tips_us[[('size', 'Male')]]

,size
sex,Male
day,
Thur,73
Fri,21
Sat,156
Sun,163


I find it a bit annoying to memorize a separate set of syntax, so I always convert it with a line of code like so (ps I wish this were in pandas core):

In [18]:
tips_us_copy = tips_us.copy()

tips_us_copy.columns = ['__'.join(col).strip() for col in tips_us.columns.values]

In [19]:
tips_us_copy

,size__Male,size__Female
day,,
Thur,73,79
Fri,21,19
Sat,156,63
Sun,163,53


You can of course repeat that operation as many times as you need to get the desired granularity of columns. 

But now let's try out the reverse operation. This is useful if somebody gives you data in pivot form.

## Stack

In [20]:
tips_us.stack()

size
day  sex         
Thur Male      73
     Female    79
Fri  Male      21
     Female    19
Sat  Male     156
     Female    63
Sun  Male     163
     Female    53

Again you can unstack either column index:

In [22]:
tips_us.stack(0)

,sex,Male,Female
day,,,
Thur,size,73,79
Fri,size,21,19
Sat,size,156,63
Sun,size,163,53


## What about Melting and Pivoting?

That is about it when it comes to stacking and unstacking. Anything you can do with melting and pivoting can be done with stacking and unstacking. Let's do a single example from pandas:

In [26]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                        'last': ['Doe', 'Bo'],
                        'height': [5.5, 6.0],
                        'weight': [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [27]:
# melt does stacking in one operation
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


To do this with stacking we just need to do it in two steps:

In [28]:
cheese.set_index(['first', 'last'], inplace=True)
cheese.stack().reset_index()

,first,last,level_2,0
0,John,Doe,height,5.5
1,John,Doe,weight,130.0
2,Mary,Bo,height,6.0
3,Mary,Bo,weight,150.0


I have used melt and pivot before, but after getting a better understanding of stack and unstack I have found them more versitile and stable than the former. So why learn both!

## Dummy Variables

There is one final way to transform the values in a column into headers, and this is called making dummy vars (well not quite, if you are interested in more ways to do it you can check out my [YT video](https://www.youtube.com/watch?v=WRxHfnl-Pcs&t=2s)).

Making a dummy variable will take all the `k` distinct values in one column and make `k` columns out of them. 

Let's look at an example below:

In [31]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [30]:
pd.get_dummies(tips.head(), columns=['sex'])

,total_bill,tip,smoker,day,time,size,sex_Male,sex_Female
0,16.99,1.01,No,Sun,Dinner,2,0,1
1,10.34,1.66,No,Sun,Dinner,3,1,0
2,21.01,3.50,No,Sun,Dinner,3,1,0
3,23.68,3.31,No,Sun,Dinner,2,1,0
4,24.59,3.61,No,Sun,Dinner,4,0,1


Notice the sex column was split into the sex_Male and sex_Female column. When the sex is female the sex_Female is 1 and 0 otherwise. And similarly for the sex_Male column.

This can be very useful for ML models and doing some types of analysis.

## Conclusion

These three ways to transform rows to columns and back again have served me quite well, and I'd be surprised if you'd need anything more than these. 

They are pretty intuitive, so you might not need to do too much practice. I actually don't know a good exercise for these guys as well - so if somebody has a good one they know of please send it over. 